In [46]:
import pandas as pd

In [51]:
df = pd.read_json("Data/train-esp.json")
df

,title,ingredients,instructions
0,"Tarta de queso de higos confitados, avellanas ...","1 1/2 tazas de Oporto leonado, 1 taza de higos...",Para los higos y la corteza: Combinar el Oport...
1,Bisque de chirivía con jengibre,"3 cucharadas de mantequilla, 2 1/2 tazas de pu...",Derretir la mantequilla en una olla grande a f...
2,Posole de pavo,"2 chiles anchos o pasilla secos, sin semillas,...",Retire las semillas de 2 chiles anchos o pasil...
3,Rabe de brócoli con bulgur y nueces,"1 taza de bulgur de grano medio*., 2 1/2 tazas...",Poner el bulgur en un bol mediano y verter 2 1...
4,Lechuga Frisée y Bibb con rábanos y cebollas d...,"1/4 de taza de yogur natural de leche entera, ...","En un bol pequeño, bata el yogur, la mayonesa,..."
...,...,...,...
9108,Natillas de vainilla con especias y Streusel d...,"1 cucharadita de gelatina sin sabor, 1 1/2 taz...",Natillas: Poner 1 cucharada de agua en un bol ...
9109,Sno-Cones de fruta fresca,1 libra de fresas frescas (aproximadamente 1 c...,"Triturar las fresas, las frambuesas y el azúca..."
9110,Orzo con jamón y queso de cabra,"1 libra de orzo (pasta con forma de arroz), 1/...",Precalentar la parrilla y engrasar ligeramente...
9111,Spiedini de cordero y romero,"5 cucharadas de zumo de limón fresco, 1/4 de t...",Preparar la barbacoa (a fuego medio-alto). Bat...


In [20]:
df_cook_varios = pd.read_csv('Data/intents_cook_others.csv')
df_cook_varios["label"] = df_cook_varios.label.map({ 'cook': 1, 'others': 0})
df_cook_varios

,Unnamed: 0,text,label
0,0,"Que hacer con 1 1/2 tazas de Oporto leonado, 1...",1
1,1,"Quiero Tarta de queso de higos confitados, ave...",1
2,2,Como preparar Posole de pavo,1
3,3,Si tengo 1/4 de taza de yogur natural de leche...,1
4,4,Como preparar Tarta de nueces a la antigua usa...,1
...,...,...,...
6571,6571,😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘😘...,0
6572,6572,Muchos besos no puedo darlos pero si decirlo,0
6573,6573,Gracias mi amorrrr,0
6574,6574,Te quiero millones,0


In [29]:
# Definir los documentos
documents = ['hola, como estás!',
                'Gana dinero, gana por la casa.',
                'Llámame.',
                'Hola, Llámame mañana?']
# Importar el contador de vectorizacion e inicializarlo
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
# Visualizar del objeto'count_vector' que es una instancia de 'CountVectorizer()'
count_vector.fit(documents)
names = count_vector.get_feature_names()

doc_array = count_vector.transform(documents).toarray()
doc_array

frequency_matrix = pd.DataFrame(data=doc_array, columns=names)
frequency_matrix

/home/david/bin/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,casa,como,dinero,estás,gana,hola,la,llámame,mañana,por
0,0,1,0,1,0,1,0,0,0,0
1,1,0,1,0,2,0,1,0,0,1
2,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0,1,1,0


In [34]:
# Dividir el conjunto de datos de entrenamiento y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_cook_varios['text'], df_cook_varios['label'], random_state=1)
print('Number of rows in the total set: {}'.format(df_cook_varios.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 6576
Number of rows in the training set: 4932
Number of rows in the test set: 1644


In [39]:
# Instantiate the CountVectorizer method
count_vector = CountVectorizer()
# Fit the training data and then return the matrix
training_data = count_vector.fit_transform(X_train)
# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test)

In [41]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

MultinomialNB()

In [44]:
predictions = naive_bayes.predict(testing_data)
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [45]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions)))
print('Recall score: ', format(recall_score(y_test, predictions)))
print('F1 score: ', format(f1_score(y_test, predictions)))

Accuracy score:  0.9884428223844283
Precision score:  0.9938650306748467
Recall score:  0.994475138121547
F1 score:  0.9941699907947222
